In [35]:
import os.path
from os import path

if not path.exists("kaggle.json"):
  raise Exception("Please upload kaggle.json. See https://www.kaggle.com/docs/api ")

! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
! unzip jigsaw-toxic-comment-classification-challenge.zip
! pip install transformers
! pip install pyyaml h5py # For saving model in h5 format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists
jigsaw-toxic-comment-classification-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  jigsaw-toxic-comment-classification-challenge.zip
replace sample_submission.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:

import pandas as pd
import tensorflow as tf
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import transformers
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

SEED = 1
np.random.seed(SEED)
tf.random.set_seed(SEED)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
train = pd.read_csv('train.csv.zip')
test = pd.read_csv('test.csv.zip')
test_labels = pd.read_csv('test_labels.csv.zip')



In [38]:
LABELS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
N_LABELS = len(LABELS)

In [39]:
# Preprocessing
def preprocessComments(comment):
    # Convert to lowercase, important for this bertmodel
    comment = comment.lower()

    # Remove leading and trailing spaces
    comment = comment.strip()

    # Remove consecutive spaces      
    comment = re.sub(r' +', ' ', comment)

    # Remove Newlines
    comment = re.sub(r'\n', ' ', comment)

    return comment

train.comment_text = train.comment_text.map(preprocessComments)
train_y = train[LABELS].values
test.comment_text = test.comment_text.map(preprocessComments)

if train['comment_text'].isnull().values.any():
  raise Exception("Missing data")
if test['comment_text'].isnull().values.any():
  raise Exception("Missing data")


In [40]:
test_filtered = pd.merge(test, test_labels)
test_filtered = test_filtered.drop(test_filtered.index[test_filtered['toxic'] == -1])
comments_list = train['comment_text'].tolist()
test_comments_list = test_filtered['comment_text'].tolist()
print(test_filtered.shape)
len(test_comments_list)

(63978, 8)


63978

In [41]:
bert_name = "bert-base-uncased"

bert_model = transformers.TFAutoModel.from_pretrained(bert_name)

tokenizer = transformers.BertTokenizerFast.from_pretrained(
    pretrained_model_name_or_path=bert_name, 
    config=transformers.BertConfig.from_pretrained(bert_name))

TOKENS_MAX_LENGTH = 120

def create_tokenizer(comments):
  return tokenizer(
    text=comments,
    padding='longest', 
    truncation='longest_first',
    max_length=TOKENS_MAX_LENGTH,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True)
  
train_tokenizer = create_tokenizer(comments_list)
test_tokenizer = create_tokenizer(test_comments_list)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [42]:
from pandas.core.common import random_state
att_mask = tf.keras.layers.Input(shape=(TOKENS_MAX_LENGTH,), name='attention_mask', dtype='int32') 
input_ids = tf.keras.layers.Input(shape=(TOKENS_MAX_LENGTH,), name='input_ids', dtype='int32')
input = {'attention_mask': att_mask, 'input_ids': input_ids}
x = bert_model.bert(input)

x = tf.keras.layers.GlobalAveragePooling1D()(x[0])
# x = tf.keras.layers.Flatten()(x[0])
x = tf.keras.layers.Dense(N_LABELS, activation='sigmoid')(x)
model = tf.keras.models.Model(input, x)

model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, decay=1e-5),
    metrics=['acc']
)

In [43]:
model.fit(
    {'attention_mask': train_tokenizer['attention_mask'], 'input_ids': train_tokenizer['input_ids']},
    train_y,
    validation_split=0.2,
    epochs=1,
    batch_size=64,
    verbose=1
)

1995/1995 [==============================] - 3353s 2s/step - loss: 0.0499 - acc: 0.9146 - val_loss: 0.0457 - val_acc: 0.9756


In [44]:
# Save model weights

!mkdir -p saved_model
model.save('saved_model/bert.h5') 


In [45]:
predictions = model.predict(
    {'attention_mask': test_tokenizer['attention_mask'], 'input_ids': test_tokenizer['input_ids']},
    batch_size=64,
    verbose=1
)

1000/1000 [==============================] - 516s 513ms/step


In [46]:
avg = 0
for i, label in enumerate(LABELS):
    print(label, ":")
    pb = predictions[:, i] >= 0.5
    score = f1_score(test_filtered[label], pb)
    print(score)
    avg += score

avg /= N_LABELS
print("Average f1-score:", avg)

toxic :
0.6588192017656797
severe_toxic :
0.36594202898550726
obscene :
0.6325704059527174
threat :
0.26732673267326734
insult :
0.6399262332872291
identity_hate :
0.4471243042671614
Average f1-score: 0.5019514844885937
